**合併數據**

In [47]:
import pandas as pd
import glob
import os

folder_path = './Training data/'
files = glob.glob(os.path.join(folder_path, '*.csv'))
data = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

In [49]:
data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.30,652.92,0.12
1,10,2024-03-01 17:14:47.000,0.0,1017.48,15.66,94.04,682.50,0.12
2,10,2024-03-01 17:15:47.000,0.0,1017.47,15.74,94.10,750.00,0.14
3,10,2024-03-01 17:16:47.000,0.0,1017.46,15.78,94.09,738.33,0.14
4,10,2024-03-01 17:17:47.000,0.0,1017.49,15.80,94.08,660.83,0.12
...,...,...,...,...,...,...,...,...
1375023,9,2024-10-18 14:27:02.000,0.0,1006.65,33.09,52.98,5280.83,6.71
1375024,9,2024-10-18 14:28:02.000,0.0,1006.67,33.07,53.42,4965.00,5.74
1375025,9,2024-10-18 14:29:02.000,0.0,1006.68,32.92,53.73,4709.17,5.13
1375026,9,2024-10-18 14:30:02.000,0.0,1006.70,32.92,53.55,4480.00,4.69


**異常值處理**

**SunLight(Lux)**

In [50]:
from sklearn.model_selection import train_test_split
filtered_data = data[data['Sunlight(Lux)'] != 117758.2]
# 特徵與標籤
X = filtered_data[['Power(mW)']]  # 發電量 (特徵)
y = filtered_data['Sunlight(Lux)']  # 光照度 (目標)
# 分割訓練與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
from sklearn.linear_model import LinearRegression

# 建立線性回歸模型
model = LinearRegression()

# 訓練模型
model.fit( X_train,y_train )

# 獲取模型係數與截距
print(f"模型係數: {model.coef_[0]}")  # 光照度對發電量的影響係數
print(f"模型截距: {model.intercept_}")  # 發電量基線


模型係數: 55.707553220003064
模型截距: 7360.636690987822


In [52]:
from sklearn.metrics import mean_squared_error, r2_score

# 預測
y_pred = model.predict(X_test)

# 計算誤差與解釋力
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 97694342.7416994
R2: 0.8661174593093676


In [53]:
# 用回歸模型補充異常光照度對應的發電量
data.loc[data['Sunlight(Lux)'] == 117758.2, 'Sunlight(Lux)'] = \
    model.predict(data[data['Sunlight(Lux)'] == 117758.2][['Power(mW)']])

In [54]:
data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.30,652.92,0.12
1,10,2024-03-01 17:14:47.000,0.0,1017.48,15.66,94.04,682.50,0.12
2,10,2024-03-01 17:15:47.000,0.0,1017.47,15.74,94.10,750.00,0.14
3,10,2024-03-01 17:16:47.000,0.0,1017.46,15.78,94.09,738.33,0.14
4,10,2024-03-01 17:17:47.000,0.0,1017.49,15.80,94.08,660.83,0.12
...,...,...,...,...,...,...,...,...
1375023,9,2024-10-18 14:27:02.000,0.0,1006.65,33.09,52.98,5280.83,6.71
1375024,9,2024-10-18 14:28:02.000,0.0,1006.67,33.07,53.42,4965.00,5.74
1375025,9,2024-10-18 14:29:02.000,0.0,1006.68,32.92,53.73,4709.17,5.13
1375026,9,2024-10-18 14:30:02.000,0.0,1006.70,32.92,53.55,4480.00,4.69


In [55]:
data.to_csv('check.csv', index= False)

# Model Training

**use Lstm and pre-processed data to train our model**

In [56]:
features = [ 'Pressure(hpa)', 'Temperature(°C)', 
            'Humidity(%)', 'Sunlight(Lux)']
X = data[features]
y = data['Power(mW)']

In [57]:
import numpy as np

def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X.iloc[i:i+time_steps].values)
        y_seq.append(y.iloc[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 10  # 每次輸入的時間步長
X_seq, y_seq = create_sequences(X, y, time_steps)

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, activation='relu', input_shape=(time_steps, len(features))),
    Dense(1)  # 預測單一數值
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

c:\Users\jeffi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 70657.9609 - mae: 105.8767 - val_loss: 35869.0742 - val_mae: 79.6039
Epoch 2/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - loss: 35950.6953 - mae: 80.0361 - val_loss: 36306.5195 - val_mae: 79.5743
Epoch 3/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 51s 2ms/step - loss: 35904.7266 - mae: 78.3049 - val_loss: 36247.8008 - val_mae: 74.1794
Epoch 4/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - loss: 35281.0625 - mae: 77.2152 - val_loss: 35137.5859 - val_mae: 75.6534
Epoch 5/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 34894.1953 - mae: 77.2888 - val_loss: 35917.3398 - val_mae: 81.5499
Epoch 6/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - loss: 34962.4023 - mae: 77.3871 - val_loss: 36067.6641 - val_mae: 84.8361
Epoch 7/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 34655.8867 - mae: 77.6954 - val_loss: 34488.5391 - val_mae: 76.4008
Epoch 8/50
27501/27501 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 345

In [ ]:
predictions = model.predict(X_test)

8069/8069 ━━━━━━━━━━━━━━━━━━━━ 7s 893us/step


array([[ 5.0898987e+02],
       [-7.4032640e+00],
       [-2.4849033e-01],
       ...,
       [-2.0511980e+00],
       [ 2.2856850e+02],
       [ 1.8376213e+02]], dtype=float32)

In [ ]:
import pandas as pd

results = pd.DataFrame({
    'Id': ['202402180900{:02d}'.format(i) for i in range(1, len(predictions) + 1)],
    'Power(mW)': predictions.flatten().round(2)
})

results.to_csv('upload.csv', index=False)
